In [ ]:
import torch
from torchsummary import summary
from torch.utils.data import random_split, DataLoader
from torchvision.datasets import CIFAR10
from torchvision.transforms import v2
from pathlib import Path
import wandb

import sys

sys.path.append("../src")

from trainer import Trainer
from module import VGGNetModule
from utils import model_size
from callbacks import OverfitCallback
from logger import WandbLogger

In [2]:
data_path = Path("../data")
logs_path = Path("../logs")

In [3]:
logger = WandbLogger(
    project_name="ImageClassification",
    config={
        "model_architecture": "ConvNet",
        "batch_size": 64,
        "epochs": 50,
        "optimizer": {
            "name": "Adam",
        },
        "train_split": 42_000,
        "val_split": 8000
    },
    logs_path=logs_path
)

In [4]:
dataset = CIFAR10(data_path, train=True, download=True, transform=v2.Compose([
    # Normalize
    v2.ToImage(),
    v2.ToDtype(torch.float32, scale=True),
    v2.Normalize(mean=(0.4914, 0.4822, 0.4465), std=(0.2023, 0.1994, 0.2010))
]))

train_dataset, val_dataset = random_split(
    dataset, [logger.config["train_split"], logger.config["val_split"]])

train_dataloader = DataLoader(
    train_dataset, batch_size=logger.config["batch_size"], shuffle=True)
val_dataloader = DataLoader(val_dataset, batch_size=logger.config["batch_size"])

Files already downloaded and verified


In [5]:
callbacks = [
    # OverfitCallback(limit_batches=2, max_epochs=200)
]

In [6]:
module = VGGNetModule()

trainer = Trainer(
    module=module,
    logger=logger,
    callbacks=callbacks,
    logs_path=logs_path,
    fast_dev_run=False,
    measure_time=True
)

In [ ]:
summary(module.model)

In [ ]:
try:
    trainer.fit(train_dataloader, val_dataloader)
except KeyboardInterrupt as e:
    print("Run stopped!")
finally:
    wandb.finish()

wandb: Using wandb-core as the SDK backend.  Please refer to https://wandb.me/wandb-core for more information.
wandb: Currently logged in as: sampath017. Use `wandb login --relogin` to force relogin


Time per epoch: 41.97 seconds
Epoch: 0, train_accuracy: 34.66, val_accuracy: 43.96
Epoch: 1, train_accuracy: 47.81, val_accuracy: 51.72
Epoch: 2, train_accuracy: 52.92, val_accuracy: 53.71
Epoch: 3, train_accuracy: 55.99, val_accuracy: 56.15
Epoch: 4, train_accuracy: 57.85, val_accuracy: 57.61
Epoch: 5, train_accuracy: 59.15, val_accuracy: 59.14
Epoch: 6, train_accuracy: 60.30, val_accuracy: 58.79
Epoch: 7, train_accuracy: 61.08, val_accuracy: 60.30
Epoch: 8, train_accuracy: 61.59, val_accuracy: 61.44
Epoch: 9, train_accuracy: 62.63, val_accuracy: 59.75
Epoch: 10, train_accuracy: 63.13, val_accuracy: 62.38
Epoch: 11, train_accuracy: 64.16, val_accuracy: 61.80
Epoch: 12, train_accuracy: 64.07, val_accuracy: 61.72
Epoch: 13, train_accuracy: 64.86, val_accuracy: 61.56
Epoch: 14, train_accuracy: 65.54, val_accuracy: 63.03
Epoch: 15, train_accuracy: 65.97, val_accuracy: 62.58
Epoch: 16, train_accuracy: 66.44, val_accuracy: 63.84
Epoch: 17, train_accuracy: 66.65, val_accuracy: 65.00
Epoch: 1

[Metrics](https://api.wandb.ai/links/sampath017/iwrrziwg)